In [1]:
import pandas as pd
import numpy as np


print("Delhi Air Quality Project - Model Training")

Delhi Air Quality Project - Model Training


In [6]:
# Loading AQI data
aqi_df = pd.read_csv('../data/raw/aqi.csv')
print("Data Loaded Successfully - Here are the columns:")
print(aqi_df.columns)

df = pd.read_csv('../data/processed/final_delhi_aqi_weather_merged_final.csv')
df['date'] = pd.to_datetime(df['date'])
print(" Data Loaded Successfully")

Data Loaded Successfully - Here are the columns:
Index(['date', 'state', 'area', 'number_of_monitoring_stations',
       'prominent_pollutants', 'aqi_value', 'air_quality_status', 'unit',
       'note'],
      dtype='object')
 Data Loaded Successfully
